Crear collection para que funcione el rekognition y extraiga de aqui las fotos

In [ ]:
import boto3
REGION = 'us-east-1'

def create_collection(collection_id):

    client=boto3.client('rekognition', region_name=REGION)

    #Create a collection
    print('Creating collection:' + collection_id)
    response=client.create_collection(CollectionId=collection_id)
    print('Collection ARN: ' + response['CollectionArn'])
    print('Status code: ' + str(response['StatusCode']))
    print('Done...')
    
def main():
    #
    ## Esta variable "collection_id" es importante
    ##
    collection_id='CollectionRekog1'
    create_collection(collection_id)

if __name__ == "__main__":
    main()  

Crear la bd y su estructura, con dynamo

In [ ]:
import boto3
REGION = 'us-east-1'

dynamodb = boto3.resource('dynamodb', region_name=REGION)

table = dynamodb.create_table(
    TableName='rekog-index',
    KeySchema=[
        {
            'AttributeName': 'RekognitionId',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'RekognitionId',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

table.meta.client.get_waiter('table_exists').wait(TableName='rekog-index')
print(table.item_count)

Crear tabla logs

In [ ]:
import boto3
REGION = 'us-east-1'

dynamodb = boto3.resource('dynamodb', region_name=REGION)

table = dynamodb.create_table(
    TableName='log-registry',
    KeySchema=[
        {
            'AttributeName': 'logId',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'logId',
            'AttributeType': 'S'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    }
)

table.meta.client.get_waiter('table_exists').wait(TableName='log-registry')
print(table.item_count)

Crear el bucket que usaremos

In [ ]:
import boto3

REGION = 'us-east-1'
client = boto3.client('s3')

s3 = boto3.client('s3', region_name=REGION)
s3.create_bucket(Bucket='image-store-rekog-abp')

In [ ]:
import boto3

REGION = 'us-east-1'
client = boto3.client('s3')
client.list_buckets()

La creacion de la funcion lambda se hace desde el navegador como en el taller 2.
(habrá que darle permisos a la funcion lambda, de todos los elementos s3, dynamo)
Y se configura un trigger para s3.
El codigo de la funcion:

In [ ]:
from __future__ import print_function

import boto3
from decimal import Decimal
import json
import urllib

print('Loading function')

dynamodb = boto3.client('dynamodb')
s3 = boto3.client('s3')
rekognition = boto3.client('rekognition')


# --------------- Helper Functions ------------------

def index_faces(bucket, key):

    response = rekognition.index_faces(
        Image={"S3Object":
            {"Bucket": bucket,
            "Name": key}},
            CollectionId="CollectionRekog1")
    return response
    
def update_index(tableName,faceId, fullName):
    response = dynamodb.put_item(
        TableName=tableName,
        Item={
            'RekognitionId': {'S': faceId},
            'FullName': {'S': fullName}
            }
        ) 
    
# --------------- Main handler ------------------

def lambda_handler(event, context):

    # Get the object from the event
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = urllib.unquote_plus(
        event['Records'][0]['s3']['object']['key'].encode('utf8'))

    try:

        # Calls Amazon Rekognition IndexFaces API to detect faces in S3 object 
        # to index faces into specified collection
        
        response = index_faces(bucket, key)
        
        # Commit faceId and full name object metadata to DynamoDB
        
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            faceId = response['FaceRecords'][0]['Face']['FaceId']

            ret = s3.head_object(Bucket=bucket,Key=key)
            personFullName = ret['Metadata']['fullname']

            update_index('rekog-index',faceId,personFullName)

        # Print response to console
        print(response)

        return response
    except Exception as e:
        print(e)
        print("Error processing object {} from bucket {}. ".format(key, bucket))
        raise e

Esta parte del codigo es para subir las fotos de las personas que deseamos que estén autorizadas para entrar. 
Primero se sube las fotos al bucket

In [5]:
import boto3
import uuid

s3 = boto3.resource('s3')

# Get list of objects for indexing
images=[('foto1.jpeg','Carlos Morán'),
#      ,('foto1.jpeg','Albert Einstein'),
#      ('image04.jpeg','Niels Bohr'),
#      ('image05.jpeg','Niels Bohr'),
#      ('image06.jpeg','Niels Bohr')
      ]

# Iterate through list to upload objects to S3   
for image in images:
    file = open(image[0],'rb')
    
    object = s3.Object('image-store-rekog-abp','index/'+ str(uuid.uuid4()) + ".jpeg")
    ret = object.put(Body=file,
                    Metadata={'FullName':image[1]}
                    )

ParamValidationError: Parameter validation failed:
Non ascii characters found in S3 metadata for key "FullName", value: "Carlos Morán".  
S3 metadata can only contain ASCII characters. 

Listar contenido insertado del bucket

In [ ]:
s3 = boto3.client('s3', region_name=REGION)
response = s3.list_objects_v2(
    Bucket='image-store-rekog-abp',
    Prefix=''
)
keys = [obj['Key'] for obj in response['Contents']]
print(keys)

Si la funcion lambda se ha disparado correctamente, con la siguiente celda podremos ver los elementos almacenados en la "collection"

In [6]:
import boto3

def list_faces_in_collection(collection_id):


    maxResults=2
    faces_count=0
    tokens=True

    client=boto3.client('rekognition', region_name='us-east-1')
    response=client.list_faces(CollectionId=collection_id,
                               MaxResults=maxResults)

    print('Faces in collection ' + collection_id)

 
    while tokens:

        faces=response['Faces']

        for face in faces:
            print (face)
            faces_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']
            response=client.list_faces(CollectionId=collection_id,
                                       NextToken=nextToken,MaxResults=maxResults)
        else:
            tokens=False
    return faces_count   
def main():

    collection_id='CollectionRekog1'

    faces_count=list_faces_in_collection(collection_id)
    print("faces count: " + str(faces_count))
if __name__ == "__main__":
    main()

Faces in collection CollectionRekog1
{'FaceId': '2671d2c7-a8af-4afd-82c6-7a1052022aa4', 'BoundingBox': {'Width': 0.29779601097106934, 'Height': 0.41804400086402893, 'Left': 0.42515701055526733, 'Top': 0.24244900047779083}, 'ImageId': 'b3935b41-4bf0-38d4-a1d7-44efdccb715c', 'Confidence': 100.0}
{'FaceId': '9b70bf58-113f-4a36-bd38-4154a095519c', 'BoundingBox': {'Width': 0.4828140139579773, 'Height': 0.4835380017757416, 'Left': 0.2533479928970337, 'Top': 0.2389259934425354}, 'ImageId': '77a9e17b-363c-3dfb-84d3-a9f447a51912', 'Confidence': 100.0}
faces count: 2


Eliminar imagenes de la collection

In [7]:
import boto3

def delete_faces_from_collection(collection_id, faces):

    client=boto3.client('rekognition', region_name='us-east-1')

    response=client.delete_faces(CollectionId=collection_id,
                               FaceIds=faces)
    
    print(str(len(response['DeletedFaces'])) + ' faces deleted:')
    for faceId in response['DeletedFaces']:
         print (faceId)
    return len(response['DeletedFaces'])

def main():

    collection_id='CollectionRekog1'
    faces=[]
    faces.append("ca3874db-a09b-4fa9-abfa-96744730812a")
    faces.append("2671d2c7-a8af-4afd-82c6-7a1052022aa4")

    faces_count=delete_faces_from_collection(collection_id, faces)
    print("deleted faces count: " + str(faces_count))

if __name__ == "__main__":
    main()

1 faces deleted:
2671d2c7-a8af-4afd-82c6-7a1052022aa4
deleted faces count: 1


## Analisis de imagen

Ahora le daremos una imagen, y buscará coincidencias en la collection, y ademas, nos devolverá de dynamo el nombre del usuario

In [ ]:
import boto3
import io
from PIL import Image

rekognition = boto3.client('rekognition', region_name='us-east-1')
dynamodb = boto3.client('dynamodb', region_name='us-east-1')
    
image = Image.open("pt1.jpeg")
stream = io.BytesIO()
image.save(stream,format="JPEG")
image_binary = stream.getvalue()


response = rekognition.search_faces_by_image(
        CollectionId='CollectionRekog1',
        Image={'Bytes':image_binary}                                       
        )
    
for match in response['FaceMatches']:
    print (match['Face']['FaceId'],match['Face']['Confidence'])
        
    face = dynamodb.get_item(
        TableName='rekog-index',  
        Key={'RekognitionId': {'S': match['Face']['FaceId']}}
        )
    
    if 'Item' in face:
        print (face['Item']['FullName']['S'])
    else:
        print ('no match found in person lookup')

In [ ]:
imgstring = ',/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgMCAgMDAwMEAwMEBQgFBQQEBQoHBwYIDAoMDAsKCwsNDhIQDQ4RDgsLEBYQERMUFRUVDA8XGBYUGBIUFRT/2wBDAQMEBAUEBQkFBQkUDQsNFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBT/wAARCADIAMgDAREAAhEBAxEB/8QAHQAAAQMFAQAAAAAAAAAAAAAAAgEDBAAFBgcICf/EAEAQAAEDAwIDBQUGBQMCBwAAAAEAAgMEBREGIQcSMQgTQVFhFCIycYEJFUKRobEjJDNSwRZionKCJUOSssLR4f/EABsBAAMAAwEBAAAAAAAAAAAAAAABAgMEBQYH/8QAKhEBAAICAQQBBAICAwEAAAAAAAECAxEEEiExQQUTIjJRFGFxsQYVwfD/2gAMAwEAAhEDEQA/AO5G9FzHVGOiEnGqoA2pg4EAQQBtVlI29EJKEAuN0AqAJqpOziAIdEAoTISoKQkiArCqAohMKA2SIiCId0EBwTBtwygLK0rRbZwdEAYVQBhMDCANqAMKiE1NIggFygFCEyMHZUQ0AQQChOAJUmS4QCYQFKoCkwpIpIUJIRhMBO6AacEBYgVots40oA2pwDgKoDBQBgoAwgDHVVCdCTJSAXPiUwiVt5pbfG58srWhvUk9EbLTV+su1Rw70M2X701TbqaSME902XvZD6BrcklLZ6amq/tI+GbZyynfd6loP9RlE1rf+Tgf0RsaZnoTtucONa1EdPFfY6Gd4/p3FphOfLmO36pxJTDeto1Vb7zTRzU9RHNE/dskbg5p+RCqJRpeGPa8ZaQR5hUQkAhCoKx4JgmEkqTJSAAhANuCQY8CtJunGoScaVUA40pgYKANpQBgoA2lOCkapIHyNjaS44A8UBo/tBdpqw8HNKVdZLMypuIJZT0TJBzzP6Y9B5pH48vNHij2tdf8T31TKm7y222THeio3FjC3yJG5RouppnvTXTOc5xd4kjcn5kplvaTziAANjG/UuQe9HYLjgkNeGY8RgZT0nqZjonjrrXh3VxS6f1JWUYY7+h3hfER5Fh90/kq0ibO2+zj9oky/Xek0/rukjt9VM4RxXSmOInuPQPZ+H5g4+SfgRO3etFWxV9OyaF4ex4yCEwkKgRBKwhJBsmFFAAQkDZCYY2FotuTjSgHGlWDjSgHGlAG1AGCghg7ISIFWGhu15x3ZwL4by3KOIVNdUu9npoi7A5yDgn0GMo1vsN67vH/AFxxHvWv75PdLxWS1VRLI55ycAZPgPBZenTFNplY++524LsZ6+gS0ezn3iyP3A0BjfDplGjmdGWXEzv5QBv1J6J9KNjqLjyhrWEkj6BEQmZVEHPbzl3J6g4VaKU2gqRDK33i8g7OzuEpgRL0c7BnavqLtUU+gtTVj6mbHLb6uU5cQB/TcT5AbH6KWTy78a4OGR0KsioLakyIQgKI2QAFANuQGMNWi25ONcnEA41UDjSgHWlAGPBAGEAQKEyou5WlUTzf+1F1MyouenbR3n8WNj5e7LjnBOCcdP1yqr5Teezz77wueW5OPJZ2vC626xXG5kCmpnv5umB1WG1618y26Yb3/GGUUPCy7XBhaKGYVAG7cbLD/IpHtuRwstvS0XjQlfaJTDJE72jxYBjb0z1+iy1y1t3hrZONfHOphYXUUkEg71hGOjSssTEtWaTCaR/AyR8gkNBB73lby8vyCraNMr0Rfq3Td9o7lb5ZIaimkbIx7DghwOQR+SiVw9y+EOsma/4b6fvzHZNbRxyv9HFu4/PKuO4lmKaVICkBSABwQDbkBivgtKG2caVQONKAcaUA6CgDBQBgoBQUFIsAqoS8cftDKytqe01qCGqfI6OBkLIGuGA1hYDgemSd1loxXaJ0ZpeS/wB3gpmtLi5wz6LHmyxjrNpZuNhnLeKw7R4ecLaG126maYW5aMkkb5Xjs/Jte0zt9AwcemKkRENp0Gk6NoGIGf8ApWrGS0+27FYR9Q8KbXfoCypoI5mnfLm7g+hW3jy2r3iWLJjpeNWhz5xW7OLKONtVb4yyLJEuGklvqF2MHLnWrPP8n4+tp3Rgdo4A19X3jTFI9zHYbJEOdjh9Oi2Z5UNSPju3dc3dme8hkpZCI8ZA77LM/REcuGC3AmPDXlZYKnTde6jqY+4kjdgghb9bReNw5V8c451L1t7Cuonaj7N+m5XOBMBmpseXI8gD8lsV8MFnQKpCkBSApLQC7qloG3p6DEgtNtnG9UAbSgHWlAOtQDg6IAggFyhMlDtlZPLb7UTSZtvGqy3po/h3O1tzt+KN5af0IV1RZpHs+Uja3WsTOXDWt5vU+a5fyM6xOz8VG8ztq3QRx4Dfh26rx8vbwyG3tDzzcyIiVQvTY3vaC1xwPDC2K7KYJVUPexlr2Nc09QR1WaJmGKYWe3aJihqXSNgjY1xyOUYWaLTLDaq+z2CJkZxG3nxnmwssS0rQ5D7X2l47NUUFzhjbE6oJjIH4sDJK6/DtM7q8/wA+sRqzt77PG0z2zsw2KSc59rqaiojHkwvIH7Fdmvhw5dLKkkQFIBUALigGnIDEQVpNs4CgHGlAONKAdaUA60oBcoCiUILlWHAv2p9PBV2PQ1wYed0NTU07j82td/8AEpxPdNoc59l7Rcs1ZU6jkOIGZpom+Z6k/suH8ll8Yo/y9F8ThmN5p/w2Bq/tCstVXPb7JTGomgeWSTvHukjqGjx+a0cPB646ry6mbnxjnppG2P2/tb11vrGtrrTL3WfecwEfXot//r6a7S1a/KzE/dV0bwq4z2nXtO72N5LwAXMeMOb81pZME451Ls4uRXNG6tj1FZTuc1ufe64WHoZtrTqPidp/SlMfb62ClDBkmV4Gy2KYpt4a2TJWkbtOmDxdqfRNbyhlxa92cFrButj+NePTl25WKZ7S0x2v65mrZ9HstrvaIKxj3QGPcPcXNaAPXdb3Dr09W3H59urp09M+EOjYuHnDHTGnYo2xfd9vhhe1o25+Uc5+riV2YcNl+UBRKATKAXKAElANuKAxEFaTbG1yAcaUA40oB0FAONKCECgtkJVQSNcqr2S3VU/XuonPx54BKJnUbXSvXaK/t5pcabbeOMunbpLXPc6aKqM9I/fAIz7uOgGDjIXAxciaZeqZ8vo/P+Mxfxfo441MeP8A7+0vgTaTQ8KLXG2PllfzueMYPMXEFafNvvPMuTwadOCsLNqOzS2G5Q0drt0VP7Q/MlZJDlkOfH/8V4cnX+UsuTDqPsjuxnTel9UXq+T019l9nt7WyOdUOhYY5cD3A0DzP7rodWOI7ObGPPNvujs2BofStTp50dZQQNoy93LIwN5ebHiubkz+pl2seKK+IdDUtrfWWT2rmxM1mxHmsdcm2e8dMOTuN2h6Q3equE9tqL1WhwDoO9fhgIJB5W742XU4+WYjUTqHD5OKL/dMbQdCcPrDU0Npq7xpdtrguTzDBPTzScweMfE13TxwehwVsZMl6xuLbaGPBW3mumyjwxi0zxL4W22PvK2ghuclc9h35Wh8eG7+bsfmsuDLHe0sWTizlnor4jcvTQHZdh57ZcpltW6CKEgQhB7IUKCQgMOBWk2xgoA2uQDrXII41yAdaUJECnAIXKgaqIm1VPLC/wCCRpYfkRhPW4EWmsxMenE140pJS+22gSGMQVErZANs8rjt+y8ff7LzE+n16MkcjFXJ+4hi3C2Rp0tSRjAMcsrHDyIkdlYuTH37cLBrWv7lsg2mjudMIqmIPYRncLVr2nbd6dop0XbAx4ZGfq7ZbH1OyYxShz26G2hkbRhud1rWmbTts0pEM103dhNb+5/CPdys+Ms1NxslfoOG61Qqy9zZCMbHbHyWeJmHMmJ9JMHD62Nh/iwtnLCHBrxnBHiFl+pM9mraspVq05HeeIdgEcQNTTzQvBx0YHuLv2H5Lf48Ta1Yj9sFrRixZLz+ph1YAvTvCiAQSsKwUKQpIEIQqAOCDYUCtJtjBQDjSgHA5CdnGlURxp2QB52TBC5ACSqS5x4z6SFt1nU1ri9tHc2d4OXwkAAdj16H1yvNfIY+jJ1R4l774fl/U40Yp81/05m4MXQ8t2onEl1Pcpxv1wXnwWpyo/Gf6Y+PbvaP7lvS1csjWnJO2wXOdmll2kp2xsBDeXPgE5ZolrrWV6o7ZXuNZMYooWcwjaCXOO/gN/BVSk38L3FY3LItE8QbFPQRNLHtc8F2Q0jYemMhbtMU18tXJkifDO7PeYLtRtno3mSnJwCQQR6bpX7SwV0vbYiGcw6Y3Sr3YMmohO4L0zrnxOvlXuYbfBHCPIPLc4/5n9F6Dg0+7qea+Rzaw9Ee5dANK7jy4kQC4VBQCkFwkFOCAbIQqGDArSbgwcoA29UFJ1pVQk41MHGlAETsgEJynCdhLlRLHqvS9u1fa3UVwjc5gPMySN3K+N3m0rHlw0zV6bwz8fk5ONfrxy86K6mj4WdoLV+n5HuFKajvonSOyTG9ocCegzv+i4XNwdNYivp6DhciclptbzLeWn6+MwtkactIyDnK4Fo09FSzGtWcdLHpq4voqioLqiP4msGeVZcfHyZY3EdhblY8c6tLRPE7i/WatuUEVmonCQAcry3mfk+Hp4LrcfjRjjd5amfnWvquKC1HEHVlqtjIIaGopq0BzXTdwQXY8c4W1FKb8tS2bNrvDaXB3tMW37iitV9a6muLc88obgOOVpcjj23urNi5lZjV/LpOx6hpr1YoqyB4kje3Id0Wnjid6leW+42yjsx0ks1u1VfpJe8iul2k9naDkNijAjH5lrv0Xr+HTpxxP7eL5uSbZNfpvJjlvuedygCygESkCUghQDbkGwUFaTdEEAbShMnWlURxpTB0FALnZCQl2ycEbc5XBSZe9XCZeeP2hln/ANO8U7FqSEOZFXUDaeocwdHMe7l/MfstHPEWt0ujxpmtOr1tiHDfi7JWUbKOSQMkLHNMhP8Aled5HG6Z3D0vG5MWjUp1l4VRavr6q4TsMbDIS2Vw96T138Fi/kziiKw2q8euW02szC38L6i0uPJWjkOwdGxjHD57Kq5+vy7WHFhpDMaTTdynjFI24TynAxzBrtvI7rPEx5Z7xiiGP664AwV1kqbjKYWV0QMje5iawuHkSOpSnkTWdQ89ycWK87rCTS6yl0zo636at0jaq818jaalY34iZCANh5E7/JPBinLk25efLGOmnbHCvSbdB6HtVkGDJTRBsrx+N/4nfUr19K9FYrDx97ddpszWKRWUJLXIAwUBWd0AoOFIISkAOKAwQFaTeECgDaU4TJ0FUQwUA41yEyUuQQXOVAy96tMmXyKky5U7YWmabWdXFbKpgcx9CC13i13O7BC4fPyTizVtH6em+Mw1zce9bft5yU9dX6L1FLbqiTuXU0pBLx1Hh9Ct2a1zU6o9uZF74Mk1n06i4JcWo79E+lq5mRd3hrD0C8zzOLOKeqHqOFyoyxqWztWadqLxbnVNLVuY8ROMYYTgnHXbqtPFbpnu61o6o7NfcDqC/XW51LK+8VEM0MhHLI87f7Suvmmuo6YcnDF431y3jxMv7NG8P7hUVMwa4QFrCT1dhaNKTkvFTzZOis2lhPYU4f1eu9TVnEi+xvdR0TjTWlsnwukxh8gHkBsD5k+S9lgwVxx2eK5Ge2SdS71hf0W40k2F6FJbHKTPAqgLKklZ3SCj0QAOKAwIOWk3hByCkbXKoSca5MHA5AFzIKVF6Egc/ZUDL34BydlUJam4rdobTnDKopbdLI+43usPLTW2ib3k8p8w3wH+5xARa8Y6za06iFVpbJaKVjcy1BrLU9Xra5srqunFNMIQww8/NydTjI2PXwXkOVyY5OTqr4h7vg8WeLi6bT3nu5c498KjeKn7zoox7U1uH425x1H1W9wuT0R0W8OZ8hxPqT118tB2e712lrvt3kMsbsFjtsFdq9K5avP0yXw337b90Z2hpoaOJldUumMTce87wzk7Lk24MRbdYd/F8nqurys9n42VP+q7hVU0/c0ss7pGtPqep/MrZnjRFIiWt/Om15mPDKKe9ah7Q+pI7MaiaOzteHzPAy2Ng/yfBY60px/u9la9uTPT6ekHCWgtNi0PabTaO5bTUVO2Hu4cYaQN+njnK7mL8IeezRrJaI/bPYXrKxRCdC7opk0yJySoSGOVQNDyjRKRolZUgDkBr8OWk3hhyCEHYTgtDa9URwPQBc6ATnQlbr1qC32Cjlqa+qipYI28znyODQ0eZJ6JwPDi7tD9u1lBLPZdCiOrfjlfdHbxNP8AsH4z6nb5rZpj9y1MmbXarVXZiqjrXUOor9dqh1wvjeQGoqHc0gY7OcE9BkLzfznVE0rHh6n/AI/0zGS0/l/43T3n/iVQwjPKR+y87SOz16LerIyup5Nstx5LYrOmtkrtpTWPB+2aje41EJjqegnj+MeS6eHkXp4ns4/I4tMneY7tbVnZj1A+eV9qqoqmEg4MnuO+S6teZXX3Q4t+BeJ+2ezLtFdlKpbXRG813uYDjHCOXO2cZPTdYr8zfasLx8KY72l1ToTRVs0TaWU1vpY6eNoBJaN3HzJ6laFrzady69KRSNQ5UsfaF1Rwf15fa2wVjJIKqsmbNS1ILon4e7ldgEYPqF7XHWLYqb/UPnmbJNM9+n9y7D7OHbgs/E2oltWqW02nbqxrXRSPnxDP5hpdjB9CfkovjmveGfHli/aezq613SluMTZKaeOZjhkOY7IIWBsrvGdkjSGlOAMFUmS5QSkALikGuw9aDeOByEiDkGUOVQJgYemlbr1qm2aepJam4VkNNDGOZ75HhoaPUnYfVIOY+LfbwsOnWTUmlYfvusGW98HFkDT/ANXV3/aPqtiuG1vPZq35Fa9q93E3E/j5q3ipVuN7uUrqfPMyigPLAz/tHX5nJWzWkV8NK+S1/LXDpn1HvE/RZIY2fcD+Ih4c65gq5XH7tqx7PVDyaTs76Hf81zfkOL/JwzEeY7w6/wAZy/4meJn8Z7S7E71slwkmjcHxyBrmuadnDHULwsRrtL6XFtxuF3hHfQ48+qyRBSxm62sNnJxkZ2ytirXtGy2GpFJVmKSIuBGxws8Ne1Y0zG3Re0vbIW4ztv5KmDSRrG7MsGl7nXOdyMpqaSUn5NJWTHXrvFY9seW8Y6TafTzgrZTO98shySS9x8yV7yI1EQ+YzM2tMo9BI9zDLzFvM8nb0SjuqWw+G/H/AFzwpuzJLBep4qcEPfRzPL6eQDwLDsM+YwVFqxbyyUyWr4l6GcOftAtAajoaVt7knsVcY296Joi+IOwM4c3O2fMBa84rR4blc9J8uh9IcTNMa5pGVNjvdFc4nDOaadr8fMA5H1WLUx5Z4mJ8MqBBGfBMFyglZQCEoDWzXrm7bxxsioDD0BCvF9o7BQS1ldUMp4ImlznyODQAOpJPQeqCcqcWe3lZLRBU0WlmfedZu1tS4FsDT556u+mB6rZphtP5dmpfk1r2r3lxdxE4xao4iVrp73eZq1mcx07Tyws+TBsPn1W5WlaeIc++S1/ylg0sz58c7ic+HgrYzDG91IR59EHs6RynKCA7rsg4dLdnXijHdqBumbnMBXUzf5OR53lj/s+bfD0+S8r8lw+i31qR2ny9x8Pz/qV/j5J7x4/uG/rdVd3JyHcLh6em2dvFOWDvA3LQFlqxzKHbTTSFrtubyWeGOzK6BoOA1uye2vMNEdq3iM222RmlaSUe112H1LW/+XEDkA/9RH5A+a7fxvH67/VnxDzPy/JjHj+jXzPn/DkS4PJjLR0PVeks8jVIgYIbfH4bkpx4E95MUZ72SplPQN5QlHeVT2gcVQe78nN6FGxpcbFqO66brYrjarjVW+uhdzMnppnMe0/MFLyrcx4dZ8LvtGNb2anpKLUFHR31kPK19QQYpnt8yQeXPryqIxRbwyfXtXzD0K4U8UbNxc0dRagss3PBO3EkTiOeF4+Jjh4ELXtWazqW5S8XjqhmKlewkoLbV4f6rlbdEQkwnstMU4n8TrXwt0rU3m5y8rIxiOMbukd4NaPEq6xN51CLWikdVnnJxz7T2peLMr6V5NusvN7tHE4nvPIyO/F8unouljwxj7+3Ky57ZO3iGlKmV0rmuccn5LZ01jcjeYIIjmjlaQjRlDOdvXcIggud72PJKTF3eRzeCIBKepnttXDVUsroKmFwfHIw4LSPEJWrFo1PhdL2paLVnUw6n4RccqXVzIKG5yx0l6ZgFrjhs/q319F5Ll8G2Geqnev+nveB8lXkxFLzq3+3QdXB3tqZL1y3K51Yda092P2otFWWujIA8cK57Dyh8UuMFs4XWR27ai7StPs9MD4+Z8gFs8TjX5V9R49y5vO5ePh03bzPiHEWob9W6mvFXdLhO6esqXl8jz+w9PAL22OlcVYpXxD5zly2z3nJee8rHNGHu3RMphKqD/JxtA69FfovZigZilqPmphUmm4DyFKjkLiQ4BBjtchZUv8AyTrPdF47N/8AZ47Ql54H6lZVUb3VNomc1tbb3O92Vv8AcPJw8D9CrvWLwjHknHL1e0VrK2690xb79aZxUUFbEJY3jqPMEeBB2I9FozE1nUurFotG4XslI2qg5cd1NKdKGNLnHAAySnsaedHaz4xycQNbz2+lmJtFre6GJrTs942c/wDwPr5rq8bH016p8y43Ky9VumPEOdah7XE83Q+K3WlCK5hLC0HJG4KDEz32Z8QgpK5oMJ8wjQ2GncN9+qDkT2hw3RMCBNGICkDbogWjKZmHQOa4OaS1wOQ5vUJa2cTpsbS3aE15pKkipWXMXGijGGw1zBJgeXN8X6rRvwcN53rX+HUxfKcjF26tx/bJavtX6vrabuqaittDKRgzMic53/IkLBHxeHf3TMtq3zmfWq1iGs7nebjqC4y19zrJa6tmOXyynJPoPT0XXx46Yq9FI1Dz+bLkz3m+SdyiO328lcsRuRmN1KoSZMNpmucOg2Vek+0eiH8rKf7iphU+UZw5d1EskDpm5LiiBKqDHPK4/wBwwnXym3heaOYljR+J5z9As0MOnf32cnE6WeG96LqpuaNrfb6Nrj8O4bI0fm0/msOaNxFm1x7d5q7f5lqN5qnmXHdVg3G/WJ0Nwt1Ddmu5Zo6YxxH/AHu91v6lZMdeu8VY8luik2eU1yuD5Kpz3O5snLif3Xeednuh1DCQeXcHdVpKNFOGyAPHKeiDk4P4NRj8LkyPFnLG4eZVaRvuhxHlkx5FY2VLlZ7uU0qALoCEjFHHzx774QJkUTMgjG4VFMnDAxw3aPVNOw9ywOa1jcOPVAOj+ocDIaMBAhUbebmKQkM+wA9UHCq08tIwDqnPgo8gi9yhPqUoVPlGlGYyscrHR7xP88JwUko2fw3kf3J1TZOhm7tsjwenuM+aybQ3d2WtbyaB4yaSrWvxHJWR0U4z1ZKeR3/uz9FVo3WYKk9N4l67ErnOu1PzLjuq5t7deoXW3hfbrcx+DX1w5hnq1jSf3wtzixvJtpcudY9ft561B5nHGxK67jQWjk5gYX7kdPknCZj2ZnhLXlh8OiYE/wDi0rXg5czYoCSJOema7zCqET5QnsLXh46HYqJhkhPaBJCqTPkMA2c0/RIxQjEhagpONbyz+hTHpIczHy805RCPEMc8h+QUmciGGF2dzug4FH8LsIEmJ9y35pScCuA9xo9FVhAHsxTRtU+h7Nuiyz6KVQSjGIZSfAJQJBTuLafbq52yqpWSIsCRjfwRe8fUqolMsj09dXWq/wBkqhnniroZsDb4Xh3+Fk89kR27vbq3XCK6W6lrIXh8NRE2VjmnILXAEfutCY068TuNtXly4rrONPtBLmfatJUIcQ0RTzY9SWj/AAujxI8y5vMn8YcXyEErpOYZc4jD2/E05QSVK72uASs+MdQsnpHhHpZB3pYfheMFSaTSj+WkaerHYVQUjbEJYnAfRItips8rmnYhEHJyNvvNI890D0p3uVP1S8SPR+ZmJGnzVJSKjAgyBvhNKK8ckTWdMpKENhg42CkxQ7sdhOClHn2e0eqmVQcqgHPHlhOShWAY2JelFLMtd8kaLaLG3lgnHRQqTLZxFC09TvgeacEepxgNBOc+84qoKV0tkne1jah3wMyGf5Kywxy9euynqCXUvZ80VWzuLpPYu5Lj1Ije5g/RoWpf8pdHFP2QAvXBdxxN9oI1w1DpR/4TRyj8nj/7XR4ntzOb5q4/lcTlw3x1XSc0G+A9pz6IA4pe5eHt/pv2PoU4lMwbqSGTczSQfEJnCdRnvWVJHjhyqGOx6kIMJ8wUyOMjDZQfNIytHLVgeB6I9j0CrBbKDhEiqRJl0Mb00ycn3jYD4oSnN0rd6q50tC2gmFVPGZYonjlL2D8Qz4bH8ipmWWKyttfSzW+plgnjdFNG4sexw3aR4KdnrQad3uHwVQmTFQ7D2qZVB+c7tPXITkoE3ZgSgSL5+KZIdV/Dgnx4qFLR32JQOpaAAFO16T6cOlG+zD1PmqhMri+oMEAYwe+4crWhZWLXd7B9lq2R2Xs86Cpos8v3XHKSfFz8vP6uK1beXRp2rD//2Q=='

In [ ]:
img_uncode = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAMCAgMCAgMDAwMEAwMEBQgFBQQEBQoHBwYIDAoMDAsKCwsNDhIQDQ4RDgsLEBYQERMUFRUVDA8XGBYUGBIUFRT/2wBDAQMEBAUEBQkFBQkUDQsNFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBT/wAARCADIAMgDAREAAhEBAxEB/8QAHQAAAQMFAQAAAAAAAAAAAAAAAgEDBAAFBgcICf/EAEAQAAEDAwIDBQUGBQMCBwAAAAEAAgMEBREGIQcSMQgTQVFhFCIycYEJFUKRobEjJDNSwRZionKCJUOSssLR4f/EABsBAAMAAwEBAAAAAAAAAAAAAAABAgMEBQYH/8QAKhEBAAICAQQBBAICAwEAAAAAAAECAxEEEiExQQUTIjJRFGFxsQYVwfD/2gAMAwEAAhEDEQA/AO5G9FzHVGOiEnGqoA2pg4EAQQBtVlI29EJKEAuN0AqAJqpOziAIdEAoTISoKQkiArCqAohMKA2SIiCId0EBwTBtwygLK0rRbZwdEAYVQBhMDCANqAMKiE1NIggFygFCEyMHZUQ0AQQChOAJUmS4QCYQFKoCkwpIpIUJIRhMBO6AacEBYgVots40oA2pwDgKoDBQBgoAwgDHVVCdCTJSAXPiUwiVt5pbfG58srWhvUk9EbLTV+su1Rw70M2X701TbqaSME902XvZD6BrcklLZ6amq/tI+GbZyynfd6loP9RlE1rf+Tgf0RsaZnoTtucONa1EdPFfY6Gd4/p3FphOfLmO36pxJTDeto1Vb7zTRzU9RHNE/dskbg5p+RCqJRpeGPa8ZaQR5hUQkAhCoKx4JgmEkqTJSAAhANuCQY8CtJunGoScaVUA40pgYKANpQBgoA2lOCkapIHyNjaS44A8UBo/tBdpqw8HNKVdZLMypuIJZT0TJBzzP6Y9B5pH48vNHij2tdf8T31TKm7y222THeio3FjC3yJG5RouppnvTXTOc5xd4kjcn5kplvaTziAANjG/UuQe9HYLjgkNeGY8RgZT0nqZjonjrrXh3VxS6f1JWUYY7+h3hfER5Fh90/kq0ibO2+zj9oky/Xek0/rukjt9VM4RxXSmOInuPQPZ+H5g4+SfgRO3etFWxV9OyaF4ex4yCEwkKgRBKwhJBsmFFAAQkDZCYY2FotuTjSgHGlWDjSgHGlAG1AGCghg7ISIFWGhu15x3ZwL4by3KOIVNdUu9npoi7A5yDgn0GMo1vsN67vH/AFxxHvWv75PdLxWS1VRLI55ycAZPgPBZenTFNplY++524LsZ6+gS0ezn3iyP3A0BjfDplGjmdGWXEzv5QBv1J6J9KNjqLjyhrWEkj6BEQmZVEHPbzl3J6g4VaKU2gqRDK33i8g7OzuEpgRL0c7BnavqLtUU+gtTVj6mbHLb6uU5cQB/TcT5AbH6KWTy78a4OGR0KsioLakyIQgKI2QAFANuQGMNWi25ONcnEA41UDjSgHWlAGPBAGEAQKEyou5WlUTzf+1F1MyouenbR3n8WNj5e7LjnBOCcdP1yqr5Teezz77wueW5OPJZ2vC626xXG5kCmpnv5umB1WG1618y26Yb3/GGUUPCy7XBhaKGYVAG7cbLD/IpHtuRwstvS0XjQlfaJTDJE72jxYBjb0z1+iy1y1t3hrZONfHOphYXUUkEg71hGOjSssTEtWaTCaR/AyR8gkNBB73lby8vyCraNMr0Rfq3Td9o7lb5ZIaimkbIx7DghwOQR+SiVw9y+EOsma/4b6fvzHZNbRxyv9HFu4/PKuO4lmKaVICkBSABwQDbkBivgtKG2caVQONKAcaUA6CgDBQBgoBQUFIsAqoS8cftDKytqe01qCGqfI6OBkLIGuGA1hYDgemSd1loxXaJ0ZpeS/wB3gpmtLi5wz6LHmyxjrNpZuNhnLeKw7R4ecLaG126maYW5aMkkb5Xjs/Jte0zt9AwcemKkRENp0Gk6NoGIGf8ApWrGS0+27FYR9Q8KbXfoCypoI5mnfLm7g+hW3jy2r3iWLJjpeNWhz5xW7OLKONtVb4yyLJEuGklvqF2MHLnWrPP8n4+tp3Rgdo4A19X3jTFI9zHYbJEOdjh9Oi2Z5UNSPju3dc3dme8hkpZCI8ZA77LM/REcuGC3AmPDXlZYKnTde6jqY+4kjdgghb9bReNw5V8c451L1t7Cuonaj7N+m5XOBMBmpseXI8gD8lsV8MFnQKpCkBSApLQC7qloG3p6DEgtNtnG9UAbSgHWlAOtQDg6IAggFyhMlDtlZPLb7UTSZtvGqy3po/h3O1tzt+KN5af0IV1RZpHs+Uja3WsTOXDWt5vU+a5fyM6xOz8VG8ztq3QRx4Dfh26rx8vbwyG3tDzzcyIiVQvTY3vaC1xwPDC2K7KYJVUPexlr2Nc09QR1WaJmGKYWe3aJihqXSNgjY1xyOUYWaLTLDaq+z2CJkZxG3nxnmwssS0rQ5D7X2l47NUUFzhjbE6oJjIH4sDJK6/DtM7q8/wA+sRqzt77PG0z2zsw2KSc59rqaiojHkwvIH7Fdmvhw5dLKkkQFIBUALigGnIDEQVpNs4CgHGlAONKAdaUA60oBcoCiUILlWHAv2p9PBV2PQ1wYed0NTU07j82td/8AEpxPdNoc59l7Rcs1ZU6jkOIGZpom+Z6k/suH8ll8Yo/y9F8ThmN5p/w2Bq/tCstVXPb7JTGomgeWSTvHukjqGjx+a0cPB646ry6mbnxjnppG2P2/tb11vrGtrrTL3WfecwEfXot//r6a7S1a/KzE/dV0bwq4z2nXtO72N5LwAXMeMOb81pZME451Ls4uRXNG6tj1FZTuc1ufe64WHoZtrTqPidp/SlMfb62ClDBkmV4Gy2KYpt4a2TJWkbtOmDxdqfRNbyhlxa92cFrButj+NePTl25WKZ7S0x2v65mrZ9HstrvaIKxj3QGPcPcXNaAPXdb3Dr09W3H59urp09M+EOjYuHnDHTGnYo2xfd9vhhe1o25+Uc5+riV2YcNl+UBRKATKAXKAElANuKAxEFaTbG1yAcaUA40oB0FAONKCECgtkJVQSNcqr2S3VU/XuonPx54BKJnUbXSvXaK/t5pcabbeOMunbpLXPc6aKqM9I/fAIz7uOgGDjIXAxciaZeqZ8vo/P+Mxfxfo441MeP8A7+0vgTaTQ8KLXG2PllfzueMYPMXEFafNvvPMuTwadOCsLNqOzS2G5Q0drt0VP7Q/MlZJDlkOfH/8V4cnX+UsuTDqPsjuxnTel9UXq+T019l9nt7WyOdUOhYY5cD3A0DzP7rodWOI7ObGPPNvujs2BofStTp50dZQQNoy93LIwN5ebHiubkz+pl2seKK+IdDUtrfWWT2rmxM1mxHmsdcm2e8dMOTuN2h6Q3equE9tqL1WhwDoO9fhgIJB5W742XU4+WYjUTqHD5OKL/dMbQdCcPrDU0Npq7xpdtrguTzDBPTzScweMfE13TxwehwVsZMl6xuLbaGPBW3mumyjwxi0zxL4W22PvK2ghuclc9h35Wh8eG7+bsfmsuDLHe0sWTizlnor4jcvTQHZdh57ZcpltW6CKEgQhB7IUKCQgMOBWk2xgoA2uQDrXII41yAdaUJECnAIXKgaqIm1VPLC/wCCRpYfkRhPW4EWmsxMenE140pJS+22gSGMQVErZANs8rjt+y8ff7LzE+n16MkcjFXJ+4hi3C2Rp0tSRjAMcsrHDyIkdlYuTH37cLBrWv7lsg2mjudMIqmIPYRncLVr2nbd6dop0XbAx4ZGfq7ZbH1OyYxShz26G2hkbRhud1rWmbTts0pEM103dhNb+5/CPdys+Ms1NxslfoOG61Qqy9zZCMbHbHyWeJmHMmJ9JMHD62Nh/iwtnLCHBrxnBHiFl+pM9mraspVq05HeeIdgEcQNTTzQvBx0YHuLv2H5Lf48Ta1Yj9sFrRixZLz+ph1YAvTvCiAQSsKwUKQpIEIQqAOCDYUCtJtjBQDjSgHA5CdnGlURxp2QB52TBC5ACSqS5x4z6SFt1nU1ri9tHc2d4OXwkAAdj16H1yvNfIY+jJ1R4l774fl/U40Yp81/05m4MXQ8t2onEl1Pcpxv1wXnwWpyo/Gf6Y+PbvaP7lvS1csjWnJO2wXOdmll2kp2xsBDeXPgE5ZolrrWV6o7ZXuNZMYooWcwjaCXOO/gN/BVSk38L3FY3LItE8QbFPQRNLHtc8F2Q0jYemMhbtMU18tXJkifDO7PeYLtRtno3mSnJwCQQR6bpX7SwV0vbYiGcw6Y3Sr3YMmohO4L0zrnxOvlXuYbfBHCPIPLc4/5n9F6Dg0+7qea+Rzaw9Ee5dANK7jy4kQC4VBQCkFwkFOCAbIQqGDArSbgwcoA29UFJ1pVQk41MHGlAETsgEJynCdhLlRLHqvS9u1fa3UVwjc5gPMySN3K+N3m0rHlw0zV6bwz8fk5ONfrxy86K6mj4WdoLV+n5HuFKajvonSOyTG9ocCegzv+i4XNwdNYivp6DhciclptbzLeWn6+MwtkactIyDnK4Fo09FSzGtWcdLHpq4voqioLqiP4msGeVZcfHyZY3EdhblY8c6tLRPE7i/WatuUEVmonCQAcry3mfk+Hp4LrcfjRjjd5amfnWvquKC1HEHVlqtjIIaGopq0BzXTdwQXY8c4W1FKb8tS2bNrvDaXB3tMW37iitV9a6muLc88obgOOVpcjj23urNi5lZjV/LpOx6hpr1YoqyB4kje3Id0Wnjid6leW+42yjsx0ks1u1VfpJe8iul2k9naDkNijAjH5lrv0Xr+HTpxxP7eL5uSbZNfpvJjlvuedygCygESkCUghQDbkGwUFaTdEEAbShMnWlURxpTB0FALnZCQl2ycEbc5XBSZe9XCZeeP2hln/ANO8U7FqSEOZFXUDaeocwdHMe7l/MfstHPEWt0ujxpmtOr1tiHDfi7JWUbKOSQMkLHNMhP8Aled5HG6Z3D0vG5MWjUp1l4VRavr6q4TsMbDIS2Vw96T138Fi/kziiKw2q8euW02szC38L6i0uPJWjkOwdGxjHD57Kq5+vy7WHFhpDMaTTdynjFI24TynAxzBrtvI7rPEx5Z7xiiGP664AwV1kqbjKYWV0QMje5iawuHkSOpSnkTWdQ89ycWK87rCTS6yl0zo636at0jaq818jaalY34iZCANh5E7/JPBinLk25efLGOmnbHCvSbdB6HtVkGDJTRBsrx+N/4nfUr19K9FYrDx97ddpszWKRWUJLXIAwUBWd0AoOFIISkAOKAwQFaTeECgDaU4TJ0FUQwUA41yEyUuQQXOVAy96tMmXyKky5U7YWmabWdXFbKpgcx9CC13i13O7BC4fPyTizVtH6em+Mw1zce9bft5yU9dX6L1FLbqiTuXU0pBLx1Hh9Ct2a1zU6o9uZF74Mk1n06i4JcWo79E+lq5mRd3hrD0C8zzOLOKeqHqOFyoyxqWztWadqLxbnVNLVuY8ROMYYTgnHXbqtPFbpnu61o6o7NfcDqC/XW51LK+8VEM0MhHLI87f7Suvmmuo6YcnDF431y3jxMv7NG8P7hUVMwa4QFrCT1dhaNKTkvFTzZOis2lhPYU4f1eu9TVnEi+xvdR0TjTWlsnwukxh8gHkBsD5k+S9lgwVxx2eK5Ge2SdS71hf0W40k2F6FJbHKTPAqgLKklZ3SCj0QAOKAwIOWk3hByCkbXKoSca5MHA5AFzIKVF6Egc/ZUDL34BydlUJam4rdobTnDKopbdLI+43usPLTW2ib3k8p8w3wH+5xARa8Y6za06iFVpbJaKVjcy1BrLU9Xra5srqunFNMIQww8/NydTjI2PXwXkOVyY5OTqr4h7vg8WeLi6bT3nu5c498KjeKn7zoox7U1uH425x1H1W9wuT0R0W8OZ8hxPqT118tB2e712lrvt3kMsbsFjtsFdq9K5avP0yXw337b90Z2hpoaOJldUumMTce87wzk7Lk24MRbdYd/F8nqurys9n42VP+q7hVU0/c0ss7pGtPqep/MrZnjRFIiWt/Om15mPDKKe9ah7Q+pI7MaiaOzteHzPAy2Ng/yfBY60px/u9la9uTPT6ekHCWgtNi0PabTaO5bTUVO2Hu4cYaQN+njnK7mL8IeezRrJaI/bPYXrKxRCdC7opk0yJySoSGOVQNDyjRKRolZUgDkBr8OWk3hhyCEHYTgtDa9URwPQBc6ATnQlbr1qC32Cjlqa+qipYI28znyODQ0eZJ6JwPDi7tD9u1lBLPZdCiOrfjlfdHbxNP8AsH4z6nb5rZpj9y1MmbXarVXZiqjrXUOor9dqh1wvjeQGoqHc0gY7OcE9BkLzfznVE0rHh6n/AI/0zGS0/l/43T3n/iVQwjPKR+y87SOz16LerIyup5Nstx5LYrOmtkrtpTWPB+2aje41EJjqegnj+MeS6eHkXp4ns4/I4tMneY7tbVnZj1A+eV9qqoqmEg4MnuO+S6teZXX3Q4t+BeJ+2ezLtFdlKpbXRG813uYDjHCOXO2cZPTdYr8zfasLx8KY72l1ToTRVs0TaWU1vpY6eNoBJaN3HzJ6laFrzady69KRSNQ5UsfaF1Rwf15fa2wVjJIKqsmbNS1ILon4e7ldgEYPqF7XHWLYqb/UPnmbJNM9+n9y7D7OHbgs/E2oltWqW02nbqxrXRSPnxDP5hpdjB9CfkovjmveGfHli/aezq613SluMTZKaeOZjhkOY7IIWBsrvGdkjSGlOAMFUmS5QSkALikGuw9aDeOByEiDkGUOVQJgYemlbr1qm2aepJam4VkNNDGOZ75HhoaPUnYfVIOY+LfbwsOnWTUmlYfvusGW98HFkDT/ANXV3/aPqtiuG1vPZq35Fa9q93E3E/j5q3ipVuN7uUrqfPMyigPLAz/tHX5nJWzWkV8NK+S1/LXDpn1HvE/RZIY2fcD+Ih4c65gq5XH7tqx7PVDyaTs76Hf81zfkOL/JwzEeY7w6/wAZy/4meJn8Z7S7E71slwkmjcHxyBrmuadnDHULwsRrtL6XFtxuF3hHfQ48+qyRBSxm62sNnJxkZ2ytirXtGy2GpFJVmKSIuBGxws8Ne1Y0zG3Re0vbIW4ztv5KmDSRrG7MsGl7nXOdyMpqaSUn5NJWTHXrvFY9seW8Y6TafTzgrZTO98shySS9x8yV7yI1EQ+YzM2tMo9BI9zDLzFvM8nb0SjuqWw+G/H/AFzwpuzJLBep4qcEPfRzPL6eQDwLDsM+YwVFqxbyyUyWr4l6GcOftAtAajoaVt7knsVcY296Joi+IOwM4c3O2fMBa84rR4blc9J8uh9IcTNMa5pGVNjvdFc4nDOaadr8fMA5H1WLUx5Z4mJ8MqBBGfBMFyglZQCEoDWzXrm7bxxsioDD0BCvF9o7BQS1ldUMp4ImlznyODQAOpJPQeqCcqcWe3lZLRBU0WlmfedZu1tS4FsDT556u+mB6rZphtP5dmpfk1r2r3lxdxE4xao4iVrp73eZq1mcx07Tyws+TBsPn1W5WlaeIc++S1/ylg0sz58c7ic+HgrYzDG91IR59EHs6RynKCA7rsg4dLdnXijHdqBumbnMBXUzf5OR53lj/s+bfD0+S8r8lw+i31qR2ny9x8Pz/qV/j5J7x4/uG/rdVd3JyHcLh6em2dvFOWDvA3LQFlqxzKHbTTSFrtubyWeGOzK6BoOA1uye2vMNEdq3iM222RmlaSUe112H1LW/+XEDkA/9RH5A+a7fxvH67/VnxDzPy/JjHj+jXzPn/DkS4PJjLR0PVeks8jVIgYIbfH4bkpx4E95MUZ72SplPQN5QlHeVT2gcVQe78nN6FGxpcbFqO66brYrjarjVW+uhdzMnppnMe0/MFLyrcx4dZ8LvtGNb2anpKLUFHR31kPK19QQYpnt8yQeXPryqIxRbwyfXtXzD0K4U8UbNxc0dRagss3PBO3EkTiOeF4+Jjh4ELXtWazqW5S8XjqhmKlewkoLbV4f6rlbdEQkwnstMU4n8TrXwt0rU3m5y8rIxiOMbukd4NaPEq6xN51CLWikdVnnJxz7T2peLMr6V5NusvN7tHE4nvPIyO/F8unouljwxj7+3Ky57ZO3iGlKmV0rmuccn5LZ01jcjeYIIjmjlaQjRlDOdvXcIggud72PJKTF3eRzeCIBKepnttXDVUsroKmFwfHIw4LSPEJWrFo1PhdL2paLVnUw6n4RccqXVzIKG5yx0l6ZgFrjhs/q319F5Ll8G2Geqnev+nveB8lXkxFLzq3+3QdXB3tqZL1y3K51Yda092P2otFWWujIA8cK57Dyh8UuMFs4XWR27ai7StPs9MD4+Z8gFs8TjX5V9R49y5vO5ePh03bzPiHEWob9W6mvFXdLhO6esqXl8jz+w9PAL22OlcVYpXxD5zly2z3nJee8rHNGHu3RMphKqD/JxtA69FfovZigZilqPmphUmm4DyFKjkLiQ4BBjtchZUv8AyTrPdF47N/8AZ47Ql54H6lZVUb3VNomc1tbb3O92Vv8AcPJw8D9CrvWLwjHknHL1e0VrK2690xb79aZxUUFbEJY3jqPMEeBB2I9FozE1nUurFotG4XslI2qg5cd1NKdKGNLnHAAySnsaedHaz4xycQNbz2+lmJtFre6GJrTs942c/wDwPr5rq8bH016p8y43Ky9VumPEOdah7XE83Q+K3WlCK5hLC0HJG4KDEz32Z8QgpK5oMJ8wjQ2GncN9+qDkT2hw3RMCBNGICkDbogWjKZmHQOa4OaS1wOQ5vUJa2cTpsbS3aE15pKkipWXMXGijGGw1zBJgeXN8X6rRvwcN53rX+HUxfKcjF26tx/bJavtX6vrabuqaittDKRgzMic53/IkLBHxeHf3TMtq3zmfWq1iGs7nebjqC4y19zrJa6tmOXyynJPoPT0XXx46Yq9FI1Dz+bLkz3m+SdyiO328lcsRuRmN1KoSZMNpmucOg2Vek+0eiH8rKf7iphU+UZw5d1EskDpm5LiiBKqDHPK4/wBwwnXym3heaOYljR+J5z9As0MOnf32cnE6WeG96LqpuaNrfb6Nrj8O4bI0fm0/msOaNxFm1x7d5q7f5lqN5qnmXHdVg3G/WJ0Nwt1Ddmu5Zo6YxxH/AHu91v6lZMdeu8VY8luik2eU1yuD5Kpz3O5snLif3Xeednuh1DCQeXcHdVpKNFOGyAPHKeiDk4P4NRj8LkyPFnLG4eZVaRvuhxHlkx5FY2VLlZ7uU0qALoCEjFHHzx774QJkUTMgjG4VFMnDAxw3aPVNOw9ywOa1jcOPVAOj+ocDIaMBAhUbebmKQkM+wA9UHCq08tIwDqnPgo8gi9yhPqUoVPlGlGYyscrHR7xP88JwUko2fw3kf3J1TZOhm7tsjwenuM+aybQ3d2WtbyaB4yaSrWvxHJWR0U4z1ZKeR3/uz9FVo3WYKk9N4l67ErnOu1PzLjuq5t7deoXW3hfbrcx+DX1w5hnq1jSf3wtzixvJtpcudY9ft561B5nHGxK67jQWjk5gYX7kdPknCZj2ZnhLXlh8OiYE/wDi0rXg5czYoCSJOema7zCqET5QnsLXh46HYqJhkhPaBJCqTPkMA2c0/RIxQjEhagpONbyz+hTHpIczHy805RCPEMc8h+QUmciGGF2dzug4FH8LsIEmJ9y35pScCuA9xo9FVhAHsxTRtU+h7Nuiyz6KVQSjGIZSfAJQJBTuLafbq52yqpWSIsCRjfwRe8fUqolMsj09dXWq/wBkqhnniroZsDb4Xh3+Fk89kR27vbq3XCK6W6lrIXh8NRE2VjmnILXAEfutCY068TuNtXly4rrONPtBLmfatJUIcQ0RTzY9SWj/AAujxI8y5vMn8YcXyEErpOYZc4jD2/E05QSVK72uASs+MdQsnpHhHpZB3pYfheMFSaTSj+WkaerHYVQUjbEJYnAfRItips8rmnYhEHJyNvvNI890D0p3uVP1S8SPR+ZmJGnzVJSKjAgyBvhNKK8ckTWdMpKENhg42CkxQ7sdhOClHn2e0eqmVQcqgHPHlhOShWAY2JelFLMtd8kaLaLG3lgnHRQqTLZxFC09TvgeacEepxgNBOc+84qoKV0tkne1jah3wMyGf5Kywxy9euynqCXUvZ80VWzuLpPYu5Lj1Ije5g/RoWpf8pdHFP2QAvXBdxxN9oI1w1DpR/4TRyj8nj/7XR4ntzOb5q4/lcTlw3x1XSc0G+A9pz6IA4pe5eHt/pv2PoU4lMwbqSGTczSQfEJnCdRnvWVJHjhyqGOx6kIMJ8wUyOMjDZQfNIytHLVgeB6I9j0CrBbKDhEiqRJl0Mb00ycn3jYD4oSnN0rd6q50tC2gmFVPGZYonjlL2D8Qz4bH8ipmWWKyttfSzW+plgnjdFNG4sexw3aR4KdnrQad3uHwVQmTFQ7D2qZVB+c7tPXITkoE3ZgSgSL5+KZIdV/Dgnx4qFLR32JQOpaAAFO16T6cOlG+zD1PmqhMri+oMEAYwe+4crWhZWLXd7B9lq2R2Xs86Cpos8v3XHKSfFz8vP6uK1beXRp2rD//2Q=='

In [ ]:
import boto3
import io
import base64
import uuid
from datetime import datetime


def update_index(tableName,faceId, fullName, conf):
    response = dynamodb.put_item(
        TableName=tableName,
        Item={
            'logId': {'S': str(uuid.uuid4())},
            'RekognitionId': {'S': faceId},
            'FullName': {'S': fullName},
            'DateTime': {'S': str(datetime.now())},
            'Confidence': {'S': str(conf)}
            }
        )

rekognition = boto3.client('rekognition', region_name='us-east-1')
dynamodb = boto3.client('dynamodb', region_name='us-east-1')
    
imgdata = base64.b64decode(imgstring)
stream = io.BytesIO(imgdata)
image_binary = stream.getvalue()


response = rekognition.search_faces_by_image(
        CollectionId='CollectionRekog1',
        Image={'Bytes':image_binary}                                       
        )
    
for match in response['FaceMatches']:
    print (match['Face']['FaceId'],match['Face']['Confidence'])
        
    face = dynamodb.get_item(
        TableName='rekog-index',  
        Key={'RekognitionId': {'S': match['Face']['FaceId']}}
        )
    
    if 'Item' in face:
        update_index('log-registry',match['Face']['FaceId'],face['Item']['FullName']['S'], match['Face']['Confidence'])
        print (face['Item']['FullName']['S'])
    else:
        print ('Personal no autorizado. Acceso Denegado')

In [ ]:
    jsonify(response, 200)

def jsonify(obj=None, statusCode=200):
    response = {
        'statusCode': statusCode,
        'headers': DEFAULT_HEADERS
    }
    if obj is not None:
        response['headers']['Content-Type'] = 'application/json'
        response['body'] = json.dumps(obj)

    return response